In [1]:
import time
import pandas as pd
import re
import citation_normalisation as cn
import importlib
importlib.reload(cn)

2021-06-21 10:23:23 DESKTOP-LKA1KPD metapub.config[9136] WARNING NCBI_API_KEY was not set.


<module 'citation_normalisation' from 'C:\\Users\\Otto Brinkhaus\\Documents\\Arbeit\\COCONUT\\citation_normalisation\\citation_normalisation.py'>

In [5]:
# COCONUT examples
test_list = [
    '10.7164/antibiotics.33.463',
    '10.1186/s13321-020-00465-0',
    #'Kluepfel,J. Antibiotics 25,(1972),109',
    #'21939219'
    #'1293891283912839128391283912893'
    #'Harborne, The Handbook of Natural Flavonoids, 2, (1999), 517,Isoflavonoids and neoflavonoids'
    #'Ratnayake,J.Nat.Prod.,71,(2008),403',
    #'Probst,Helvetica Chimica Acta,64,(1981),2056',
    #'Braz Filho,Phytochem.,19,(1980),2003'
]
references = []
for test_ID in test_list:
    reference = cn.get_structured_reference(test_ID)
    references.append(reference)
    

In [8]:
print(references)

# Print exemplary dict
for key in references[1].keys():
    print(key + ' - ' + str(references[1][key]))

[{'indexed': {'date-parts': [[2020, 3, 26]], 'date-time': '2020-03-26T07:49:33Z', 'timestamp': 1585208973633}, 'reference-count': 0, 'publisher': 'Japan Antibiotics Research Association', 'issue': '5', 'content-domain': {'domain': [], 'crossmark-restriction': False}, 'short-container-title': ['J. Antibiot.'], 'published-print': {'date-parts': [[1980]]}, 'DOI': '10.7164/antibiotics.33.463', 'type': 'journal-article', 'created': {'date-parts': [[2012, 3, 1]], 'date-time': '2012-03-01T03:31:37Z', 'timestamp': 1330572697000}, 'page': '463-467', 'source': 'Crossref', 'is-referenced-by-count': 21, 'title': ['Antibiotics from basidiomycetes. X. Scorodonin, a new antibacterial and antifungal metabolite from Marasmius scorodonius (Fr.) Fr.'], 'prefix': '10.7164', 'volume': '33', 'author': [{'given': 'TIMM', 'family': 'ANKE', 'sequence': 'first', 'affiliation': []}, {'given': 'JUTTA', 'family': 'KUPKA', 'sequence': 'additional', 'affiliation': []}, {'given': 'GEORG', 'family': 'SCHRAMM', 'sequen

In [ ]:
for reference_dict in references:
    reference_str = cn.create_normalized_reference_str(reference_dict)
    print(reference_str)

In [2]:
# Read data from file
coconut_references = pd.read_csv('./coconut_references.csv')
unstructured_references = coconut_references['citationDOI']
COCONUT_IDs = coconut_references['coconut_id']

ID_ref_tuples = zip(COCONUT_IDs, unstructured_references)

# Get everything into the right format and filter empty reference lists
ID_ref_tuples = [(tup[0], eval(tup[1])) # the references are read as str and need to be converted to lists
                for tup in ID_ref_tuples
                if len(eval(tup[1])) != 0 # Don't include empty reference lists
                if eval(tup[1]) != ['NA']] # Don't include reference list with 'NA' as only element


In [119]:
sub_pat = '[A-Z]\.'
pat = '[((A-Z)\.)]+'
test_str = "Peter Lustig A."

print(re.search(pat, test_str))

<re.Match object; span=(0, 1), match='P'>


In [285]:
## Check regex for typical pattern: 
# Pattern: Author,? (et al.?), Journal, issue, (year), page(-page)?
# Commas are mandatory separators here.

# Examples that are supposed to be matched
ref_str_1 = 'Haba,Phytochem.,68,(2007),1255'
ref_str_2 = 'El-Sayed,Phytochem.,30,(1991),2442'
ref_str_3 = 'Fujita,J.Nat.Prod.,49,(1986),1122-1125'
ref_str_4 = 'Kim, et al., Chem Pharm Bull, 52, (2004), 1466'
ref_str_5 = 'Lansky et al.,J.Ethnopharmacol.,19,(2007),177-206'
ref_str_6 = 'Imperato,Chim.Ind.(Milan),71,(1989),86'
ref_str_7 = 'Cole,R.J.et al.,Can.J.Microbiol.,20(1974),1159'
ref_str_8 = 'Mathews.,J. Biol. Chem.,241(21),(1966),5008'
ref_str_9 = " Fang,Chung Ts'ao Yao,12,(1981),1"
ref_str_10 = 'N.V.Thu,Pharmazie,26,(1971),504'
ref_str_11 = 'Ruan,Yun-Nan Chih Wu Yen Chiu,13,(1991),225'
ref_str_12 = 'Peng J.-P.,Phytochem.,41,(1996),283-285'
ref_str_13 = ' Hussain,J.Nat.Prod.51.,(1988),809'

ref_str_issue_missing_1 = 'Bondarenko,Khim.Prir.Soedin,(1983),243'
ref_str_year_typo_1 = 'Haba,Phytochem.,68,82007),1255'
ref_str_year_typo_2 = 'Ingham,Phytochem.,15,819769,1489'
ref_str_issue_missing_1 = 'Bondarenko,Khim.Prir.Soedin,(1983),243'


# Define regular expression
authors = '([A-Z\-]\.\s?)*[a-zA-Z\-\s]+,?([A-Z]\.\-?\s?)?\,?\s?([A-Z]\.\s?)?\.?(\s?\,|et al\.?,?)\s?'
journal = '[a-zA-Z\-\'\s\.]+(\s?\([A-Za-z]+\)\s?)?(,\s?Suppl.)?,?\.?\s?'
issue_volume = '(No\.?\s?)?\d+[A-Ea-e]?(\s?\(\d+\))?\.?,?\s?'
year = '\((20|1\d)\d\d\),?\s?'
typo_year = '(8(20|1\d)\d\d\),?\s?|8(20|1\d)\d\d9,?\s?|\((20|1\d)\d\d9,?\s?)'
pages = '\d+(\-\d+)?'
general_pattern = '^\s?' + authors + journal + issue_volume + year + pages + '\.?$'
general_pattern_without_issue = '^' + authors + journal + year + pages + '\.?$'
# Typo that occurs regularily (see ref_str_year_typo_$N)
general_pattern_with_year_typo = '^' + authors + journal + issue_volume + typo_year + pages + '\.?$' 


#print(re.search(authors+journal+issue_volume, ref_str_13))

assert re.search(general_pattern, ref_str_1)
assert re.search(general_pattern, ref_str_2)
assert re.search(general_pattern, ref_str_3)
assert re.search(general_pattern, ref_str_4)
assert re.search(general_pattern, ref_str_5)
assert re.search(general_pattern, ref_str_6)
assert re.search(general_pattern, ref_str_7)
assert re.search(general_pattern, ref_str_8)
assert re.search(general_pattern, ref_str_9)
assert re.search(general_pattern, ref_str_10)
assert re.search(general_pattern, ref_str_11)
assert re.search(general_pattern, ref_str_12)
assert re.search(general_pattern, ref_str_13)


assert re.search(general_pattern_with_year_typo, ref_str_year_typo_1)
assert re.search(general_pattern_with_year_typo, ref_str_year_typo_2)



assert not re.search(general_pattern, ref_str_neg_1)
assert not re.search(general_pattern, ref_str_neg_2)

In [286]:
# Define regex for a rarer structured reference pattern
ref_str_1 = 'J_Agric_Food_Chem_2016_64_(21):4255-4263'
ref_str_2 = 'J_Nat_Prod_2015_78_(4):730-735'
ref_str_3 = "Phytochemistry_2003;64:285-291"

journal = '[A-Za-z\_]'
year = '(20|1\d)\d\d'
gap = '[_:;]'
issue_volume = '\d+([_:;]\(\d+\))?'
pages = '\d+(\-\d+)?'

underscore_pattern = '^\s?' + journal + gap + year + gap + issue_volume + gap + pages + '$'

assert re.search(underscore_pattern, ref_str_1)
assert re.search(underscore_pattern, ref_str_2)
assert re.search(underscore_pattern, ref_str_3)


In [294]:
# Regex for J.Chem.Soc. references 

ref_str_1 = 'Gunasekera,J.Chem.Soc.,Perkin 1,(1975),2447'
ref_str_2 = 'Locksley,J.Chem.Soc.,C,(1971),1332'

# Define regular expression
authors = '([A-Z\-]\.\s?)*[a-zA-Z\-\s]+,?([A-Z]\.\-?\s?)?\,?\s?([A-Z]\.\s?)?\.?(\s?\,|et al\.?,?)\s?'
journal = 'J\.\s?Chem\.\s?Soc\.,\s?'
whatever = '(C|Perkin\s?1),'
year = '\((20|1\d)\d\d\),?\s?'
pages = '\d+(\-\d+)?'

j_chem_soc_pattern = '^\s?' + authors + journal + whatever + year + pages + '$'

assert re.search(j_chem_soc_pattern, ref_str_1)
assert re.search(j_chem_soc_pattern, ref_str_2)

In [287]:
# Check regex for the Handbook of Natural Flavonoids (Harborne)

# Type 1 
harborne_str_1 = 'Harborne, The Handbook of Natural Flavonoids, 2, (1999), 115,Chalcones,dihydrochalcones and aurones'
harborne_str_2 = 'Harborne, The Handbook of Natural Flavonoids, 1, (1999), 181.Flavonols'
harborne_str_3 = 'Harborne, The Handbook of Natural Flavonoids, 1, (1999), 3.Flavone O-glycosides, John Wiley & Son'

# Regex for Type 1
harborne_pattern_1= '^Harborne,\s?The Handbook of Natural Flavonoids,\s?[12],\s+\(1999\),\s?\d+\s?[,\.][a-zA-Z\s\,\-]+(,\s?John Wiley\s?&\s?Son)?\.?$'

# Type 2
harborne_str_4 = 'Harborne,Phytochemical Dictionary Second Edition,Taylor and Francis,(1999),Chapter54\.?'

# Regex for Type 2
harborne_pattern_2 = 'Harborne,\s?Phytochemical Dictionary Second Edition,\s?Taylor and Francis,\s?\(1999\),\s?Chapter\s?\d+'

assert re.search(harborne_pattern_1, harborne_str_1)
assert re.search(harborne_pattern_1, harborne_str_2)
assert re.search(harborne_pattern_1, harborne_str_3)
assert re.search(harborne_pattern_2, harborne_str_4)

In [288]:
# How many references are there and what type of references are we dealing with?

total_number = 0 # Total number of references
unique_number = 0 # Tumber of unique references
PMID_number = 0 # Number of PubMed IDs
DOI_number = 0 # Number of DOIs
general_pattern_number = 0 # Number of references that can be matched exactly be the pattern above.
general_pattern_with_year_typo_number = 0 # 81999) instead of (1999)
general_pattern_without_issue_number = 0
harborne_1_number = 0
harborne_2_number = 0
underscore_pattern_number = 0
no_digits_number = 0

no_digits_references = []
unmatched_references = []



references = []
for tup in ID_ref_tuples:
    for ref in tup[1]:
        if ref != "NA":
            total_number += 1
            if ref not in references:
                unique_number += 1
                references.append(ref)
                # Check for DOI
                if cn.contains_DOI(ref):
                    DOI_number += 1
                # Check for PMID (reference str is a number of at least 6 digits)
                elif ref.isdigit():
                    if len(ref) > 3:
                        PMID_number += 1
                # Check for specific, frequently occuring patterns
                elif re.search(general_pattern, ref):
                    general_pattern_number += 1
                elif re.search(general_pattern_with_year_typo, ref):
                    general_pattern_with_year_typo_number += 1
                elif re.search(general_pattern_without_issue, ref):
                    general_pattern_without_issue_number += 1
                elif re.search(underscore_pattern, ref):
                    underscore_pattern_number += 1
                elif 
                # Check for Harbornes Handbook of Natural Flavonoids
                elif re.search(harborne_pattern_1, ref):
                    harborne_1_number += 1
                # Check for Harbornes Phytochemical Dictionary Second Edition
                elif re.search(harborne_pattern_2, ref):
                    harborne_2_number += 1
                # Check for (useless) references that contain no digits
                elif re.search('\D+', ref).group() == ref:
                    no_digits_number += 1
                    no_digits_references.append(ref)
                else:
                    unmatched_references.append((tup[0], ref))
#print(buckingham_number)

In [291]:
non_specific_ref = unique_number - DOI_number - PMID_number - general_pattern_number - general_pattern_without_issue_number - harborne_number

print('There are {} COCONUT entries'.format(len(COCONUT_IDs)))
print('{} of them have a total of {} references ({} of them are unique).'.format(len(ID_ref_tuples), total_number, unique_number))
print('{} of them are PMIDs.'.format(PMID_number))
print('{} of them are DOIs.'.format(DOI_number))
print('Another {} of them follow a very specific pattern (as in: Haba,Phytochem.,68,(2007),1255). This would offer enough information'.format(general_pattern_number))
print('Another {} of them follow the exact same pattern but there is a typo in the year. (as in: Haba,Phytochem.,68,82007),1255)'.format(general_pattern_with_year_typo_number))
print('Another {} of them follow the same pattern but the issue is not included (as in: Haba,Phytochem.,(2007),1255). TBD if this is precise enough.'.format(general_pattern_without_issue_number))
print('Another {} of them follow a different specific pattern (as in: J_Nat_Prod_2015_78_(4):730-735).'.format(underscore_pattern_number))
print('Another {} of them follow a different pattern that only occures in J Chem Soc references (as in: Locksley,J.Chem.Soc.,C,(1971),1332)'.format(jchem_soc_number))
print('Another {} of them come from Harborne´s Handbook of Natural Flavonoids'.format(harborne_1_number))
print('Another {} of them com from Harborne´s Phytochemical Dictionary Second Edition'.format(harborne_2_number))
print('That leaves us with {} unique references that do not match a specific pattern (that we know).'.format(non_specific_ref))
print('{} of them do not contain any digit.'.format(no_digits_number))

There are 406747 COCONUT entries
70969 of them have a total of 158824 references (66151 of them are unique).
20953 of them are PMIDs.
12232 of them are DOIs.
Another 27188 of them follow a very specific pattern (as in: Haba,Phytochem.,68,(2007),1255). This would offer enough information
Another 80 of them follow the exact same pattern but there is a typo in the year. (as in: Haba,Phytochem.,68,82007),1255)
Another 2036 of them follow the same pattern but the issue is not included (as in: Haba,Phytochem.,(2007),1255). TBD if this is precise enough.
Another 117 of them follow a different specific pattern (as in: J_Nat_Prod_2015_78_(4):730-735).
Another 588 of them come from Harborne´s Handbook of Natural Flavonoids
Another 59 of them com from Harborne´s Phytochemical Dictionary Second Edition
That leaves us with 3154 unique references that do not match a specific pattern (that we know).
285 of them do not contain any digit.


In [292]:
for ref in unmatched_references[:150]:
    print(ref)

('CNP0106606', 'Cole,Handbook of Secondary Fungal Metabolites,Volume I,(2003)')
('CNP0309481', 'Yin, et al., Modern Study of Chinese Drugs and Clinical Applications (1), Xueyuan Press, Beijing, (1993).')
('CNP0309481', 'Buckingham(Executive Editor), Dictionary of Natural Products, Chapman & Hall, 1994, Vol1-7')
('CNP0309481', 'Buckingham(Executive Editor)Dictionary of Natural ProductsChapman & Hall 1995, Vol8')
('CNP0309481', 'Buckingham(Executive Editor)Dictionary of Natural ProductsChapman & Hall 1996, Vol9')
('CNP0309481', 'Buckingham(Executive Editor)Dictionary of Natural ProductsChapman & Hall 1997, Vol10')
('CNP0309481', 'Buckingham(Executive Editor)Dictionary of Natural ProductsChapman & Hall 1998, Vol11.')
('CNP0251815', 'Ohmiya,The Alkaloids,47,(1995),1,Lupine alkaloids')
('CNP0251815', 'Ji, et al., Pharmacological Action and Application of Available Composition of Traditional Chinese Medicine, Heilongjiang Science and technology Press, Heilongjiang, (1995).')
('CNP0251815', '

In [ ]:
importlib.reload(cn)

problematic_str = 'Haba,Phytochem.,68,82007),1255'
ref_dict = cn.get_structured_reference(problematic_str)

In [ ]:
ref_dict